### APD, CASE

In [17]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_case as dr_c
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [18]:
importlib.reload(train)

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE,
# ]
] + train.Metrics.STATISTICAL

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

anxiety_label_type = None
case_label_type = dr_c.SelfReports.AROUSAL

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

threshold = "fixed"
test_size = 1.0

percent_of_target_dataset = 0.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_case_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_case_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [19]:
# TRAIN ON APD AND TEST ON CASE -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dt)


random.seed(15)

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_apd_case = {}
ensemble_weights_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_apd_case[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models_apd_case, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models_apd_case, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_apd_case, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_apd_case, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_apd[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.56336557 0.56889696 0.56872613 0.57881503 0.5748592  0.54223594
 0.55785411 0.56366709 0.5705041  0.56840635 0.5438036  0.54793973
 0.55109067 0.55960456 0.56496136 0.55167657 0.5468123  0.5621077
 0.55761876 0.55911687 0.56086159 0.56728347 0.5678099  0.56549915
 0.57385374        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'sigmoid', 'probability': True}
LGB: {'max_depth': 5, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [354 126], Predictions: [0 1], [329 151]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [354 126], Predictions: [0 1], [208 272]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [354 126], Predictions: [0 1], [ 97 383]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 

In [20]:
# TRAIN ON CASE AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dt)


random.seed(13)

x_a, y_a = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_case = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_case / n_apd)) * n_case)
# augment_indices = random.sample(range(n_case), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_case_apd = {}
ensemble_weights_case = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_case_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models_case_apd, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models_case_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_case_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_case_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights_case[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.77769485 0.7817253  0.79151802 0.80249914 0.80615565 0.78295626
 0.78390142 0.79693636 0.801229   0.80874115 0.78591457 0.78981739
 0.79655108 0.80035457 0.79915789 0.79538074 0.79734075 0.80081189
 0.81213633 0.80992491 0.7997105  0.80695557 0.80513734 0.81228206
 0.81057948        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 6, 'metric': 'binary_logloss', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [572 462], Predictions: [0 1], [258 776]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [572 462], Predictions: [0 1], [509 525]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [572 462], Predictions: [0 1], [305 729]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean'

In [21]:
# TRAIN ON CASE AND TEST ON APD - SPEECH TASK ONLY
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dt)


random.seed(13)

model_phases_apd_speech = [
    "Baseline_Rest", 
    # "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd_speech, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_case = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_case / n_apd)) * n_case)
# augment_indices = random.sample(range(n_case), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_case_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_case_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models_case_apd, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models_case_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_case_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_case_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.79207071 0.80075195 0.80491284 0.80645556 0.8164615  0.80943009
 0.80151139 0.80522399 0.810812   0.81701023 0.80787257 0.80044473
 0.80633973 0.80852802 0.81643994 0.80280186 0.79730154 0.80002861
 0.80404368 0.81249287 0.81477364 0.80663963 0.80752464 0.80648661
 0.81501203        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 1, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [343 243], Predictions: [0 1], [185 401]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [343 243], Predictions: [0 1], [262 324]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [343 243], Predictions: [0 1], [140 446]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean' 'eda_me

In [22]:
# TRAIN ON CASE AND TEST ON APD - BUG TASK ONLY
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dt)


random.seed(13)

model_phases_apd_speech = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    # "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd_speech, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

# augment smaller dataset by randomly duplicating a subset of subjects
# n_case = x_a.shape[0]
# n_apd = x_b.shape[0]
# n_samples_to_augment = int((0.7 - (n_case / n_apd)) * n_case)
# augment_indices = random.sample(range(n_case), n_samples_to_augment)

# duplicates = x_a.iloc[augment_indices, :].reset_index(drop=True)
# x_a = pd.concat([x_a, duplicates], axis=0).reset_index(drop=True)
# duplicates = y_a.iloc[augment_indices, :].reset_index(drop=True)
# y_a = pd.concat([y_a, duplicates], axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models_case_apd = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models_case_apd[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models_case_apd, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models_case_apd, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models_case_apd, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models_case_apd, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.77655227 0.7902981  0.8009932  0.81332801 0.81291115 0.78901571
 0.80213744 0.81213508 0.81360515 0.81842471 0.79319514 0.80772835
 0.8120186  0.81628638 0.81706431 0.79847598 0.81078605 0.82215171
 0.81994792 0.82631717 0.79730031 0.8105376  0.82073064 0.8186816
 0.82131524        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [297 255], Predictions: [0 1], [133 419]
coef_ only available for SVC with linear kernel
SVM
Model LGB, Actual: [0 1], [297 255], Predictions: [0 1], [268 284]
LGB
['bpm', 'rmssd', 'hf_rr', 'lf_rr', 'sdnn', 'mean_SCL', 'SCR_rate', 'ecg_mean', 'ecg_median', 'ecg_std', 'ecg_var', 'eda_mean', 'eda_median', 'eda_std', 'eda_var', 'lf_hf_ratio']
Model RF, Actual: [0 1], [297 255], Predictions: [0 1], [167 385]
RF
['bpm' 'rmssd' 'hf_rr' 'lf_rr' 'sdnn' 'mean_SCL' 'SCR_rate' 'ecg_mean'
 'ecg_median' 'ecg_std' 'ecg_var' 'eda_mean'

In [23]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier

random.seed(13)

percent_of_target_dataset = 0.0
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard')
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds

In [24]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_c)
importlib.reload(dt)

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

voting_type = "weighted_avg"
# voting_type = "majority_vote"

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_apd_case[name] for name in ensemble_models}
weights = [ensemble_weights_apd[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print("Train APD, test CASE")
print(f"APD size: {x_train.shape[0]}\n{y_train.value_counts()}\nCASE size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

###############################################################################################################

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

x_a, y_a = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True, standardize=False)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True, standardize=False)
# drop subjects with noisy data
# x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

subjects = list(np.unique(x_b.loc[:, "subject"]))
train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
x_train = pd.concat([x_a, x_train_addition])
y_train = pd.concat([y_a, y_train_addition])
x_test = x_b[~x_b["subject"].isin(train_subjects)]
y_test = y_b[~y_b["subject"].isin(train_subjects)]

y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models_apd_case[name] for name in ensemble_models}
weights = [ensemble_weights_case[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
weights = np.divide(weights, len(weights))
# weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print("Train CASE, test APD")
print(f"CASE size: {x_train.shape[0]}\n{y_train.value_counts()}\nAPD size: {x_test.shape[0]}\n{y_test.value_counts()}")
print("Predictions:")
print(f"{np.array(np.unique(y_pred, return_counts=True)).T}")
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

Train APD, test CASE
APD size: 1034
0    572
1    462
Name: label, dtype: int64
CASE size: 480
0    354
1    126
Name: label, dtype: int64
Predictions:
[[  0 336]
 [  1 144]]
	Accuracy: 0.675
	Precision: 0.3958333333333333
	Recall: 0.4523809523809524
	F1-score: 0.4222222222222222
	AUC score: 0.6033091202582729
----------------------------------------
Train CASE, test APD
CASE size: 480
0    354
1    126
Name: label, dtype: int64
APD size: 1034
0    572
1    462
Name: label, dtype: int64
Predictions:
[[  0 561]
 [  1 473]]
	Accuracy: 0.5735009671179884
	Precision: 0.5221987315010571
	Recall: 0.5346320346320347
	F1-score: 0.5283422459893048
	AUC score: 0.5697635697635698
----------------------------------------
